In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/PFA' 

Mounted at /content/drive


In [62]:
import cv2
import csv
from glob import glob
from tqdm import tqdm
import numpy as np
import imutils

pathcsv = os.path.join("/content/drive/MyDrive/PFA/new_data/real_size.csv")


In [63]:
def load_data(path):
    predicted_icm_mask = sorted(glob(f"{path}/new_data/train/GT_ICM/*_0.bmp"))
    predicted_zp_mask = sorted(glob(f"{path}/new_data/train/GT_ZP/*_0.bmp"))
    predicted_te_mask = sorted(glob(f"{path}/new_data/train/GT_TE/*_0.bmp"))
    predicted_b_mask = sorted(glob(f"{path}/new_data/train/GT_blastocoel/*.bmp"))
    predicted_roi = sorted(glob(f"{path}/new_data/train/GT_full/*_0.bmp"))
    print(len(predicted_roi))
   
    return predicted_icm_mask,predicted_zp_mask, predicted_te_mask, predicted_roi,predicted_b_mask

In [64]:
def find_size(elem):
  original_height = 0
  original_width = 0
  filecsv = open(pathcsv)
  real_sizes = csv.reader(filecsv)
  for row in real_sizes:
    if row[0]==elem :
      original_height= int(row[1])
      original_width= int(row[2])

  return original_width, original_height

In [65]:
def area(image, area_type=None):
  #this part just shows the nb of pixels with a specific colour
  colours, counts = np.unique(image.reshape(-1,1), axis=0, return_counts=1)
  
  
  area = counts[1]
 
    
  return area

In [66]:
from google.colab.patches import cv2_imshow


In [71]:
def zp_thickness(image, roi):
  roi=roi.copy()
  roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(roi, 60, 255, cv2.THRESH_BINARY)[1]
  cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)

  # loop over the contours
  for c in cnts:
    # compute the center of the contour
    M = cv2.moments(c)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

  w = image.shape[1]
  h = image.shape[0]
  h_c=cX
  w_c=cY  
  width=0
  zp_right=0
  while(width<=w_c) :
      if image[h_c,width]==255 :
          zp_right=zp_right+1
      width=width+1
  
  width=w_c
  zp_left=0
  while(width<w) :
      if image[h_c,width]==255 :
          zp_left=zp_left+1
      width=width+1
  
  height=0
  zp_high=0
  while (height<=h_c) :
        if image[height,w_c]==255 :
          zp_high=zp_high+1
        height=height+1
  
  height=h_c
  zp_down=0
  while (height<h) :
        
        if image[height-1,w_c]==255 :
          zp_down=zp_down+1
        height=height+1
  
  zp_thickness=(zp_down+zp_high+zp_right+zp_left)/4
  
  
  return zp_thickness

In [74]:
import math
import csv
with open('/content/drive/MyDrive/PFA/new_data/features.csv', 'w', newline='\n') as file:
              writer = csv.DictWriter(file, fieldnames=["File Name", "ICM Area", "TE Area", "ZP Area", "Blastocyst radius", "TE Thickness", "ZP Thickness","Expansion"])
              writer.writeheader()

In [75]:
def extract(predicted_icm_mask,predicted_zp_mask, predicted_te_mask, predicted_roi_mask, predicted_b_mask):
    """Performing data augmentation"""
   
    print(len(predicted_icm_mask))
    for idx, (icm, te,zp, roi,b) in tqdm(enumerate(zip(predicted_icm_mask,predicted_zp_mask, predicted_te_mask, predicted_roi_mask,predicted_b_mask)), total=len(predicted_icm_mask)):
        
        name_mask = icm.split("/")[-1].split(".")[0]
        
        name_mask=name_mask.replace(' ICM_Mask', '')
        
        name_mask += '.bmp'
        if name_mask =='Blast_PCRM_R13-0127_0.bmp' or name_mask =='Blast_PCRM_R13-0145_0.bmp': continue
        predicted_icm = cv2.imread(icm)
        predicted_te = cv2.imread(te)
        predicted_zp = cv2.imread(zp)
        predicted_roi = cv2.imread(roi)
        predicted_b = cv2.imread(b)
        width, height = find_size(name_mask)
        predicted_icm[predicted_icm >0] =255
        predicted_te[predicted_te >0] =255
        predicted_zp[predicted_zp >0] =255
        predicted_roi[predicted_roi >0] =255
        predicted_b[predicted_b >0] =255
        icm = cv2.resize(predicted_icm, (width, height), interpolation = cv2.INTER_NEAREST)
        te = cv2.resize(predicted_te, (width, height), interpolation = cv2.INTER_NEAREST)
        zp = cv2.resize(predicted_zp, (width, height), interpolation = cv2.INTER_NEAREST)
        full = cv2.resize(predicted_roi, (width, height), interpolation = cv2.INTER_NEAREST)
        b = predicted_b
        area_zp = area(zp)
        area_te= area(te)
        area_icm = area(icm)
        full_area = area(full)
        area_b = area(b)
        radius = math.sqrt(full_area / math.pi)
        print(full_area)
        expansion = area_b / full_area
        print(expansion)
        thickness_zp = zp_thickness(zp, full)  
        thickness_te = zp_thickness(te, full)
        features=[name_mask,area_icm, area_te,area_zp, radius,thickness_zp,thickness_te, expansion]
        with open('/content/drive/MyDrive/PFA/new_data/features.csv', 'a+', newline='\n') as file:
              writer = csv.writer(file)
              writer.writerow(features)
        
        #print(area_icm, area_zp,aera_te, radius,thickness_zp,thickness_te)

In [76]:
predicted_icm_mask,predicted_zp_mask, predicted_te_mask, predicted_roi , predicted_b_mask= load_data(drive_path)
extract(predicted_icm_mask,predicted_zp_mask, predicted_te_mask, predicted_roi,predicted_b_mask)

189
189


  1%|          | 1/189 [00:01<03:18,  1.05s/it]

231333
0.38195588178080947


  1%|          | 2/189 [00:02<03:16,  1.05s/it]

190707
0.3424153282260221


...

348051
0.6608945240783678


 99%|█████████▉| 188/189 [03:31<00:01,  1.19s/it]

228645
0.6544905858426819


100%|██████████| 189/189 [03:32<00:00,  1.12s/it]

220800
0.6530298913043479
